In [1]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import coralme
from coralme.builder.main import MEBuilder, MEReconstruction, METroubleshooter
from coralme.builder.curation import MECurator, MEManualCuration


In [3]:
org = 'cljungdahlii'
organism = './{:s}/organism.json'.format(org)
inputs = './{:s}/input.json'.format(org)

### Full

In [4]:
builder = MEBuilder(*[organism, inputs])

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16


In [5]:
builder.generate_files(overwrite=True)
builder.save_builder_info()

Initiating file processing...
~ Processing files for cljungdahlii...


Checking M-model metabolites...                                            : 100.0%|██████████|   718/  718 [00:00<00:00]
Checking M-model genes...                                                  : 100.0%|██████████|   637/  637 [00:00<00:00]
Checking M-model reactions...                                              : 100.0%|██████████|   816/  816 [00:00<00:00]
Syncing optional genes file...                                             : 100.0%|██████████|  4360/ 4360 [00:00<00:00]
Looking for duplicates within datasets...                                  : 100.0%|██████████|     5/    5 [00:00<00:00]
Gathering ID occurrences across datasets...                                : 100.0%|██████████| 10367/10367 [00:00<00:00]
Solving duplicates across datasets...                                      : 0.0%|          |     0/    0 [00:00<?]
Pruning GenBank...                                                         : 100.0%|██████████|     1/    1 [00:01<00:00]
Updating Genbank file with opt

unknown metabolite 'tyr__L_e' created
unknown metabolite 'dpm_c' created
unknown metabolite '5drib_c' created


Adding manual curation of complexes...                                     : 100.0%|██████████|     3/    3 [00:00<00:00]
Getting sigma factors...                                                   : 100.0%|██████████|    25/   25 [00:00<00:00]
Getting generics from Genbank contigs...                                   : 100.0%|██████████|   108/  108 [00:00<00:00]
Getting TU-gene associations from optional TUs file...                     : 100.0%|██████████|  2904/ 2904 [00:00<00:00]
Adding protein location...                                                 : 100.0%|██████████|  5081/ 5081 [00:00<00:00]
Purging M-model genes...                                                   : 100.0%|██████████|   638/  638 [00:00<00:00]
Getting enzyme-reaction associations...                                    : 100.0%|██████████|   826/  826 [00:00<00:00]


Reading cljungdahlii done.


...

Reading iJL1678b done.
~ Running BLAST with 4 threads...


Converting Genbank contigs to FASTA for BLAST...                           : 100.0%|██████████|   108/  108 [00:00<00:00]
Converting Genbank contigs to FASTA for BLAST...                           : 100.0%|██████████|     1/    1 [00:00<00:00]


BLAST done.


Updating translocation machinery from homology...                          : 100.0%|██████████|     9/    9 [00:00<00:00]
Updating protein location from homology...                                 : 100.0%|██████████|   514/  514 [00:00<00:00]
Updating translocation multipliers from homology...                        : 100.0%|██████████|     3/    3 [00:00<00:00]
Updating lipoprotein precursors from homology...                           : 100.0%|██████████|    14/   14 [00:00<00:00]
Updating cleaved-methionine proteins from homology...                      : 100.0%|██████████|   343/  343 [00:00<00:00]
Mapping M-metabolites to E-metabolites...                                  : 100.0%|██████████|   147/  147 [00:00<00:00]
Updating generics from homology...                                         : 100.0%|██████████|    10/   10 [00:00<00:00]
Updating folding from homology...                                          : 100.0%|██████████|     2/    2 [00:00<00:00]
Updating ribosome subrea

File processing done.


In [6]:
builder.build_me_model(overwrite=False)

Initiating ME-model reconstruction...


Adding biomass constraint(s) into the ME-model...                          : 100.0%|██████████|    11/   11 [00:00<00:00]

Read LP format model from file /tmp/tmpyafyl3f4.lp
Reading time = 0.00 seconds
: 721 rows, 1652 columns, 7064 nonzeros



Adding Metabolites from M-model into the ME-model...                       : 100.0%|██████████|   747/  747 [00:00<00:00]
Adding Reactions from M-model into the ME-model...                         : 100.0%|██████████|   835/  835 [00:00<00:00]
Adding Transcriptional Units into the ME-model...                          : 100.0%|██████████|  2904/ 2904 [00:17<00:00]
Updating all TranslationReaction and TranscriptionReaction...              : 100.0%|██████████|  7654/ 7654 [00:05<00:00]
Removing SubReactions from ComplexData...                                  : 100.0%|██████████|  5735/ 5735 [00:00<00:00]
Adding ComplexFormation into the ME-model...                               : 100.0%|██████████|  5735/ 5735 [00:00<00:00]
Adding Generic(s) into the ME-model...                                     : 100.0%|██████████|    30/   30 [00:00<00:00]
Processing StoichiometricData in ME-model...                               : 100.0%|██████████|   734/  734 [00:00<00:00]


ME-model was saved in the ./cljungdahlii/ directory as MEModel-step1-cljungdahlii.pkl


Adding tRNA synthetase(s) information into the ME-model...                 : 100.0%|██████████|   252/  252 [00:00<00:00]
Adding tRNA modification SubReactions...                                   : 100.0%|██████████|    16/   16 [00:00<00:00]
Associating tRNA modification enzyme(s) to tRNA(s)...                      : 100.0%|██████████|   370/  370 [00:00<00:00]
Adding SubReactions into TranslationReactions...                           : 100.0%|██████████|  4320/ 4320 [00:01<00:00]
Adding RNA Polymerase(s) into the ME-model...                              : 100.0%|██████████|    25/   25 [00:00<00:00]
Associating a RNA Polymerase to each Transcriptional Unit...               : 100.0%|██████████|  2904/ 2904 [00:00<00:00]
Processing ComplexData in ME-model...                                      : 100.0%|██████████|   650/  650 [00:00<00:00]
Adding ComplexFormation into the ME-model...                               : 100.0%|██████████|  5766/ 5766 [00:00<00:00]
Adding SubReactions into

ME-model was saved in the ./cljungdahlii/ directory as MEModel-step2-cljungdahlii.pkl
ME-model reconstruction is done.
Number of metabolites in the ME-model is 3577 (+396.12%, from 721)
Number of reactions in the ME-model is 5898 (+614.04%, from 826)
Number of genes in the ME-model is 972 (+52.35%, from 638)


In [7]:
builder.troubleshoot(growth_key_and_value = { builder.me_model.mu : 0.001 })

The MINOS and quad MINOS solvers are a courtesy of Prof Michael A. Saunders. Please cite Ma, D., Yang, L., Fleming, R. et al. Reliable and efficient solution of genome-scale models of Metabolism and macromolecular Expression. Sci Rep 7, 40863 (2017). https://doi.org/10.1038/srep40863

~ Troubleshooting started...
  Checking if the ME-model can simulate growth without gapfilling reactions...
  Original ME-model is feasible with a tested growth rate of 0.001000 1/h
~ Final step. Fully optimizing with precision 1e-6 and save solution into the ME-model...
  Gapfilled ME-model is feasible with growth rate 0.193273 (M-model: 0.198481).
ME-model was saved in the ./cljungdahlii/ directory as MEModel-step3-cljungdahlii-TS.pkl


### Quick

In [ ]:
builder = MEBuilder(*['./{:s}/coralme-config.yaml'.format(org)])

In [ ]:
builder.build_me_model(overwrite=False, update=True, prune=False)

In [ ]:
builder.troubleshoot(growth_key_and_value = { builder.me_model.mu : 0.001 })

### Load builder

In [ ]:
builder = MEBuilder(*['./{:s}/coralme-config.yaml'.format(org)])
builder.me_model = builder.load(builder.configuration['out_directory']+'MEModel-step2-{}.pkl'.format(org))

In [ ]:
for r in builder.me_model.metabolites.query("trnaser_c")[0].reactions:
    print(r.id, r.reaction)

In [ ]:
builder.find_issue('')

### Curation notes

In [ ]:
builder.curation_notes

### Notes

In [ ]:
from coralme.builder.helper_functions import *
pandas.set_option('display.max_colwidth', None)

In [ ]:
import pandas as pd

In [ ]:
builder = MEBuilder(*['./{:s}/coralme-config.yaml'.format(org)])
builder.me_model = builder.load(builder.configuration['out_directory']+'MEModel-step2-{}.pkl'.format(org))
me = builder.me_model

In [ ]:
a = 0
for i in me.all_genes:
    if not i.functions:
        a += 1
        print(a)
        print(i.id)
        print(i.complexes)
        print(i.functions)
        print()

In [ ]:
for r in me.get('generic_Def').reactions:
    print(r.id)
    print(r.metabolites)
    print()
    break

In [ ]:
iJL965_genes = list(i for i in pd.read_csv('./clostridium/paper/iJL965_genes.txt')['id'])
len(iJL965_genes)

In [ ]:
coralme_genes = [i.id.split('RNA_')[1] for i in me.all_genes if 'dummy' not in i.id]

In [ ]:
df = pd.read_csv('./clostridium/building_data/builder_info/complexes_df.txt',index_col=0)

In [ ]:
notinijl = set(coralme_genes) - set(iJL965_genes)
df[df['genes'].str.contains('|'.join([i+'()' for i in notinijl]))]

In [ ]:
notincoral = set(iJL965_genes) - set(coralme_genes)
print(len(notincoral)) 
df[df['genes'].str.contains('|'.join([i+'()' for i in notincoral]))]

Remaining genes that are missing from coralme:
* __CLJU_c42660__ and __CLJU_c42670__ have low-quality annotation from Uniprot, and are missing from BioCyc and GenBank. These genes are not needed for ammonia transport, as CPLX-438 performs the same job.
* __CLJU_c15890__ and __CLJU_c40080__ are deadend proteins in iJL965-ME, performing no job.
* __CLJU_c41100__ is a ribosomal protein in iJL965-ME, but is hypothetical in current annotations.


### Find issue